# Prototype Connect Four battle

Pit LLMs against each other in a game of Connect Four

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import json

In [ ]:
load_dotenv(override=True)

In [ ]:
RED = 1
YELLOW = -1
EMPTY = 0
show = {EMPTY:"⚪️", RED: "🔴", YELLOW: "🟡"}
pieces = {EMPTY: "empty", RED: "red", YELLOW: "yellow"}
cols = "ABCDEFG"

In [ ]:
class Board:

    def __init__(self):
        self.cells = [[0 for _ in range(7)] for _ in range(6)]
        self.player = RED
        self.winner = EMPTY

    def __repr__(self):
        result = ""
        for y in range(6):
            for x in range(7):
                result += show[self.cells[5-y][x]]
            result += "\n"
        if self.winner:
            result += f"\n{show[self.winner]} wins\n"
        else:
            result += f"\n{show[self.player]} to play\n"
        return result

    def json(self):
        result = "{\n"
        result += '    "Column names": ["A", "B", "C", "D", "E", "F", "G"],\n'
        for y in range(6):
            result += f'    "Row {6-y}": [' 
            for x in range(7):
                result += f'"{pieces[self.cells[5-y][x]]}", '
            result = result[:-2] + '],\n'
        result = result[:-2]+'\n}'
        return result  

In [ ]:
Board()

In [ ]:
print(Board().json())

In [ ]:
def height(self, x):
    height = 0
    while height<6 and self.cells[height][x] != EMPTY:
        height += 1
    return height

def legal_moves(self):
    return [cols[x] for x in range(7) if self.height(x)<6]

def move(self, x):
    self.cells[self.height(x)][x] = self.player
    self.player = -1 * self.player

Board.height = height
Board.legal_moves = legal_moves
Board.move = move

In [ ]:
b = Board()
b.move(3)
b.move(3)
b.move(2)
b

In [ ]:
b.legal_moves()

In [ ]:
def winning_line(self, x, y, dx, dy):
    color = self.cells[y][x]
    for pointer in range(1, 4):
        xp = x + dx * pointer
        yp = y + dy * pointer
        if not (0 <= xp <= 6 and 0 <= yp <= 5) or self.cells[yp][xp] != color:
            return EMPTY
    return color

def winning_cell(self, x, y):
    for dx, dy in ((0, 1), (1, 1), (1, 0), (1, -1)):
        if winner := self.winning_line(x, y, dx, dy):
            return winner
    return EMPTY

def wins(self):
    for y in range(6):
        for x in range(7):
            if winner := self.winning_cell(x, y):
                return winner
    return EMPTY

def move(self, x):
    self.cells[self.height(x)][x] = self.player
    if winner := self.wins():
        self.winner = winner
    else:
        self.player = -1 * self.player
    return self

Board.winning_line = winning_line
Board.winning_cell = winning_cell
Board.wins = wins
Board.move = move

In [ ]:
b = Board()
b.move(2).move(3).move(2).move(3).move(2).move(3).move(2)

In [ ]:
class Player:

    def __init__(self, model, color):
        self.color = color
        self.model = model
        self.llm = OpenAI()

    def system(self, board):
        legal_moves = ", ".join(board.legal_moves())
        return f"""You are an expert player of the board game Connect 4.
Players take turns to drop counters into one of 6 columns labelled A, B, C, D, E, F.
The winner is the first player to get 4 coins in a row in a straight or diagonal line.
You are playing with the {pieces[self.color]} coins.
And your opponent is playing with the {pieces[self.color * -1]} coins.
You will be presented with the board and asked to pick a column to drop your piece.
You must pick one of the following legal moves: {legal_moves}. You must pick one of those letters.
You should respond in JSON, and only in JSON, according to this spec:

{{
    "evaluation": "brief assessment of the board",
    "threats": "any threats from your opponent or weaknesses in your position",
    "opportunities": "any opportunities to gain the upper hand or strengths in your position",
    "strategy": "the thought process behind your next move",
    "move_column": "one letter from this list of legal moves: {legal_moves}"
}}"""

    def user(self, board):
        legal_moves = ", ".join(board.legal_moves())
        return f"""It is your turn to make a move as {pieces[self.color]}.
The current board position is:

{board.json()}

Now with this in mind, make your decision. Respond only in JSON strictly according to this spec:

{{
    "evaluation": "brief assessment of the board",
    "threats": "any threats from your opponent or weaknesses in your position",
    "opportunities": "any opportunities to gain the upper hand or strengths in your position",
    "strategy": "the thought process behind your next move",
    "move_column": "one of {legal_moves} which are the legal moves"
}}

You must pick one of these letters for your move_column: {legal_moves}

"""

    def process_move(self, reply):
        print(reply)
        try:
            result = json.loads(reply)
            move = result.get("move_column") or ""
            move = move.upper()
            col = cols.find(move)
            if not (0 <= col <= 6) or board.height(col)==6:
                raise ValueError("Illegal move")
            board.move(col)
        except Exception as e:
            print(f"Exception {e}")
            board.winner = -1 * board.player
            
    
    def move(self, board):
        system = self.system(board)
        user = self.user(board)
        reply = self.llm.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": user}
            ]
        )
        self.process_move(reply.choices[0].message.content)

In [ ]:
board = Board()
red = Player("gpt-4o-mini", RED)
yellow = Player("gpt-4o-mini", YELLOW)
while not board.winner:
    red.move(board)
    print(board)
    if not board.winner:
        yellow.move(board)
        print(board)

## A fancier version

Basically the same code, but a bit more organized.. and with a Gradio UI

In [1]:
!python c4.py

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
_____
Calling mixtral-8x7b-32768
System prompt:
You are playing the board game Connect 4.
Players take turns to drop counters into one of 7 columns A, B, C, D, E, F, G.
The winner is the first player to get 4 counters in a row in any direction.
You are red and your opponent is yellow.
You must pick a column for your move. You must pick one of the following legal moves: A, B, C, D, E, F, G.
You should respond in JSON according to this spec:

{
    "evaluation": "my assessment of the board",
    "threats": "any threats from my opponent that I should block",
    "opportunities": "my best chances to win",
    "strategy": "my thought process",
    "move_column": "one letter from this list of legal moves: A, B, C, D, E, F, G"
}

You must pick one of these letters for your move_column: A, B, C, D, E, F, G
User prompt:
It is your turn to make a move as red.
Here is the current board, with r